In [ ]:
import numpy as np
import pandas as pd
import heaan
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Data preprocessing

In [ ]:
data = pd.read_csv('/Users/xialetian/Downloads/Iriss.csv')
data

In [ ]:
columns = data.columns.values
le_species = LabelEncoder()
data['label'] = le_species.fit_transform(data[columns[5]])
data = data.drop(columns[5], axis='columns')
data = data.drop(columns[0], axis='columns')
data

In [ ]:
y = data['label'].to_numpy()
X = data.loc[:, data.columns != 'label'].to_numpy()
train_data, test_data, train_label, test_label = train_test_split(X,y)
train_data.shape

## Encrypted decision tree classification model

In [ ]:
class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


def d(a, decryptor, secret_key):
    d_2 = heaan.Message()
    decryptor.decrypt(a, secret_key, d_2)

    return d_2


def dec_arr(arr, decryptor, secret_key):
    result = []
    for i in arr:
        d = heaan.Message()
        decryptor.decrypt(i, secret_key, d)
        result.append(d)

    return result


def encMess(message, context, enc_key):
    ciphertext = heaan.Ciphertext()
    encryptor = heaan.Encryptor(context)
    encryptor.encrypt(message, enc_key, ciphertext)

    return ciphertext


def data_enc(X, label, enc_key, encryptor, context):
    # encrypt label
    mess = []
    ciphertext_label = []
    num = [[0] * X.shape[1] for x in range(len(label))]
    ciphertext_data = [[heaan.Ciphertext()] * X.shape[1] for x in range(len(label))]
    count = 0
    c = 0

    # i = 0

    for ele in label:
        mess.append(heaan.Message([ele]))
        ciphertext_label.append(heaan.Ciphertext())

    if len(label) > 0:
        a = len(label)
        for l in range(a):
            encryptor.encrypt(mess[l], enc_key, ciphertext_label[l])

    for i in X:
        for j in range(X.shape[1]):
            num[count][j] = heaan.Message([i[j]])
        count += 1
        # num[count][1] = heaan.Message([i[1]])
        # encryptor.encrypt(num[count][0], enc_key, ciphertext_data[count][0])
        # encryptor.encrypt(num[count][1], enc_key, ciphertext_data[count][1])

    for _ in range(len(num)):
        # encryptor.encrypt(num[c][0], enc_key, ciphertext_data[c][0])
        # encryptor.encrypt(num[c][1], enc_key, ciphertext_data[c][1])
        for i in range(X.shape[1]):
            ciphertext_data[c][i] = encMess(num[c][i], context, enc_key)
            # ciphertext_data[c][1] = encMess(num[c][1], context, enc_key)
        c += 1

    return ciphertext_data, ciphertext_label


def MyAdd(a, b, eval):
    add_ctxt = heaan.Ciphertext()
    eval.add(a, b, add_ctxt)

    return add_ctxt


def MyMulti(a, b, keypack, eval):
    multi_ctxt = heaan.Ciphertext()
    eval.mult(a, b, keypack, multi_ctxt)

    return multi_ctxt


def MySub(a, b, eval):
    sub_ctxt = heaan.Ciphertext()
    eval.sub(a, b, sub_ctxt)

    return sub_ctxt


def MyNegate(a, eval):
    neg_ctxt = heaan.Ciphertext()
    eval.negate(a, neg_ctxt)

    return neg_ctxt


def Inverse(x, eval, encryptor, enc_key, keypack, d):
    # 0<x<2
    a0 = heaan.Ciphertext()
    b0 = heaan.Ciphertext()

    msg = heaan.Message([2])
    ctxt = heaan.Ciphertext()
    encryptor.encrypt(msg, enc_key, ctxt)
    msg_ = heaan.Message([1])
    ctxt_ = heaan.Ciphertext()
    encryptor.encrypt(msg_, enc_key, ctxt_)
    eval.sub(ctxt, x, a0)
    eval.sub(ctxt_, x, b0)
    a = [a0]
    b = [b0]
    for i in range(1, d):
        f = heaan.Ciphertext()
        a.append(f)
        b.append(f)

    for i in range(d-1):
        if a[i].get_level() < 17 or b[i].get_level() < 17:
            eval.bootstrap(a[i], keypack, a[i])
            eval.bootstrap(b[i], keypack, b[i])

        b[i+1] = MyMulti(b[i], b[i], keypack, eval)
        # add_ctxt = heaan.Ciphertext()
        a[i+1] = MyMulti(a[i], MyAdd(ctxt_, b[i+1], eval), keypack, eval)

    return a[d-1]


def Sqrt(x, eval, encryptor, enc_key, keypack, d):
    # 0<=x<=1
    b0 = heaan.Ciphertext()

    tag1 = heaan.Message([1/2])
    ctxt_tag1 = heaan.Ciphertext()
    encryptor.encrypt(tag1, enc_key, ctxt_tag1)

    tag2 = heaan.Message([1/4])
    ctxt_tag2 = heaan.Ciphertext()
    encryptor.encrypt(tag2, enc_key, ctxt_tag2)

    tag2 = heaan.Message([3])
    ctxt_tag3 = heaan.Ciphertext()
    encryptor.encrypt(tag2, enc_key, ctxt_tag3)

    msg_ = heaan.Message([1])
    ctxt_ = heaan.Ciphertext()
    encryptor.encrypt(msg_, enc_key, ctxt_)

    eval.sub(x, ctxt_, b0)
    a = [x]
    b = [b0]
    for i in range(1, d):
        f = heaan.Ciphertext()
        a.append(f)
        b.append(f)

    for i in range(d-1):
        if a[i].get_level() < 17 or b[i].get_level() < 17:
            eval.bootstrap(a[i], keypack, a[i])
            eval.bootstrap(b[i], keypack, b[i])

        bridge_a = MyAdd(MyNegate(MyMulti(b[i], ctxt_tag1, keypack, eval), eval), ctxt_, eval)
        a[i+1] = MyMulti(a[i], bridge_a, keypack, eval)
        bridge_b = MyMulti(MySub(b[i], ctxt_tag3, eval), ctxt_tag2, keypack, eval)
        power_b = MyMulti(b[i], b[i], keypack, eval)
        b[i+1] = MyMulti(power_b, bridge_b, keypack, eval)

    return a[d-1]


def Max(a, b, encryptor, enc_key, keypack, eval):
    tag1 = heaan.Message([1 / 2])
    ctxt_tag1 = heaan.Ciphertext()
    encryptor.encrypt(tag1, enc_key, ctxt_tag1)

    # x = heaan.Ciphertext()
    # y = heaan.Ciphertext()

    x = MyMulti(MyAdd(a, b, eval), ctxt_tag1, keypack, eval)
    y = MyMulti(MySub(a, b, eval), ctxt_tag1, keypack, eval)
    power_y = MyMulti(y, y, keypack, eval)
    z = Sqrt(power_y, eval, encryptor, enc_key, keypack, 100)

    return MyAdd(x, z, eval)


class DecisionTreeClassifier:

    def __init__(self, encryptor, enc_key, eval, keypack, decryptor, secret_key, context, max_depth=None):
        self.max_depth = max_depth
        self.encryptor = encryptor
        self.enc_key = enc_key
        self.eval = eval
        self.keypack = keypack
        self.decryptor = decryptor
        self.secret_key = secret_key
        self.context = context

    def enc_multi(self, a, b):
        # both a&b are encrypted
        result = heaan.Ciphertext()
        self.eval.mult(a, b, self.keypack, result)

        return result

    def enc_y(self, y):
        result = []
        num_parent = []
        fl = dec_arr(y, self.decryptor, self.secret_key)
        a = []
        for i in fl:
            a.append(int(np.array(i).real))

        a = np.array(a)
        num_parent = [np.sum(a == c) for c in range(self.n_classes_)]
        for i in num_parent:
            result.append(i)

        return result

    def enc_num(self, num):
        f = heaan.Message([num])
        result = heaan.Ciphertext()
        self.encryptor.encrypt(f, self.enc_key, result)

        return result

    def enc_power(self, num):
        return self.enc_multi(num, num)

    def Myadd(self, a, b):
        result = heaan.Ciphertext()
        self.eval.add(a, b, result)

        return result

    def Mysub(self, a, b):
        result = heaan.Ciphertext()
        self.eval.sub(a, b, result)

        return result

    def num_multi(self, a, b):
        # b is real number
        result = heaan.Ciphertext()
        self.eval.mult(a, b, result)

        return result

    def _c(self, c):
        return int(np.array(d(c, self.decryptor, self.secret_key)).real)

    def gini_(self, num_, count):
        # gini_left = 1.0 - sum(
        #     (num_left[x] / i) ** 2 for x in range(self.n_classes_)
        # )
        # for x in range(self.n_classes_):

        return self.enc_num(1.0 - sum((num_[x] / count) ** 2 for x in range(self.n_classes_)))

    def equal(self, a, b):
        c1 = np.array(d(a[0], self.decryptor, self.secret_key)).real
        c2 = np.array(d(b[0], self.decryptor, self.secret_key)).real

        return c1 == c2

    def Sqrt(self, a):
        pass

    def sor(self, X, y, idx):
        thresholds_ = []
        for i in X:
            thresholds_.append(i[idx])
        out_X = np.array(dec_arr(thresholds_, self.decryptor, self.secret_key))
        out_y = np.array(dec_arr(y, self.decryptor, self.secret_key))
        thresholds, classes = zip(*sorted(zip(out_X, out_y)))
        enc_thr, enc_class = data_enc(np.array(thresholds), np.array(classes), self.enc_key, self.encryptor, self.context)
        # print(enc_thr, enc_class)

        return enc_thr, enc_class

    def compare(self, a, b):
        c1 = np.array(d(a, self.decryptor, self.secret_key)).real
        c2 = np.array(d(b, self.decryptor, self.secret_key)).real

        return c1 < c2

    def cal_bgini(self, m_1, num_parent):
        a = heaan.Message([0])
        ctxt_add = heaan.Ciphertext()
        self.encryptor.encrypt(a, self.enc_key, ctxt_add)
        b = heaan.Message([1])
        result = heaan.Ciphertext()
        self.encryptor.encrypt(b, self.enc_key, result)

        for i in num_parent:
            multi = self.num_multi(m_1, int(i))
            power = self.enc_power(multi)
            ctxt_add = self.Myadd(power, ctxt_add)

        return self.Mysub(result, ctxt_add)

    def cal_gini(self, i, m, m_1, gini_left, gini_right):
        # (i * gini_left + (m - i) * gini_right) / m
        l = self.num_multi(gini_left, i)
        r = self.num_multi(gini_right, m - i)

        return self.enc_multi(self.Myadd(l, r), m_1)

    def midpoint(self, a, b):
        # (thresholds[i] + thresholds[i - 1]) / 2
        return self.num_multi(self.Myadd(a[0], b[0]), 1 / 2)

    def NumberSub(self, a, num):
        result = heaan.Ciphertext()
        self.eval.sub(a, num, result)

        return result

    def FindIndex(self, c):
        for i in range(0, self.n_classes_):
            index = int(np.array(d(self.NumberSub(c, i), self.decryptor, self.secret_key)).real)
            if index == 0:
                return i

    def enc_matrix(self):
        num = [[0] for x in range(self.n_classes_)]
        a = heaan.Message([0])
        ctxt_add = heaan.Ciphertext()
        self.encryptor.encrypt(a, self.enc_key, ctxt_add)
        for i in range(self.n_classes_):
            num[i] = ctxt_add

        return num

    def transfer(self, matrix):
        for i in range(len(matrix)):
            a = heaan.Message([matrix[i]])
            ctxt_add = heaan.Ciphertext()
            self.encryptor.encrypt(a, self.enc_key, ctxt_add)
            matrix[i] = ctxt_add

        return matrix

    def add_num(self, a, num):
        result = heaan.Ciphertext()
        self.eval.add(a, num, result)

        return result

    def sub_gini(self, molecule, denominator):
        # (num_left[x] / i) ** 2 for x in range(self.n_classes_)
        a = heaan.Message([0])
        ctxt_add = heaan.Ciphertext()
        self.encryptor.encrypt(a, self.enc_key, ctxt_add)
        dem = d(denominator, self.decryptor, self.secret_key)

        for x in range(self.n_classes_):
            # molecule[x] * denominator
            multi = self.enc_multi(molecule[x], denominator)
            m = d(multi, self.decryptor, self.secret_key)
            # print("multi", d(multi, self.decryptor, self.secret_key))
            multi_power = self.enc_power(multi)
            m_p = d(multi_power, self.decryptor, self.secret_key)
            # print("multi_power", d(multi, self.decryptor, self.secret_key))
            ctxt_add = self.Myadd(ctxt_add, multi_power)
            add = d(ctxt_add, self.decryptor, self.secret_key)
            # print("ctxt_add", d(multi, self.decryptor, self.secret_key))

        return ctxt_add

    def _best_split(self, X, y):
        # Need at least two elements to split a node
        self.enc_matrix()
        m = len(y)
        if m <= 1:
            return None, None

        m_1 = self.enc_num(1 / m)
        # Count of each class in the current node.
        num_parent = self.enc_y(y)

        # Gini of current node.
        # best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_gini = self.cal_bgini(m_1, num_parent)
        # print(d(best_gini, self.decryptor, self.secret_key))
        best_idx, best_thr = None, None

        flag = heaan.Message([1])
        ctxt_flag = heaan.Ciphertext()
        self.encryptor.encrypt(flag, self.enc_key, ctxt_flag)

        # Loop through all features.
        for idx in range(self.n_features_):
            thresholds, classes = self.sor(X, y, idx)
            # Sort data along selected feature.
            # thresholds_ = []
            # for i in X:
            #     thresholds_.append(i[idx])
            # thresholds, classes = zip(*sorted(zip(X[:, idx], y)))

            num_left = self.enc_matrix()
            num_right = self.transfer(num_parent.copy())
            # num_left = [0] * self.n_classes_
            # num_right = num_parent.copy()
            # self.transfer(num_parent.copy())

            for i in range(1, m):  # possible split positions
                c = classes[i - 1]
                index = self.FindIndex(c)
                # num_left[index] += 1
                # num_right[index] -= 1
                # gini_left = self.gini_(num_left, i)
                # gini_right = self.gini_(num_right, m - i)
                num_left[index] = self.add_num(num_left[index], 1)
                num_right[index] = self.NumberSub(num_right[index], 1)

                # print("num_left: ", dec_arr(num_left, self.decryptor, self.secret_key))
                # print("num_right: ", dec_arr(num_right, self.decryptor, self.secret_key))

                a = heaan.Message([1 / i])
                b = heaan.Message([1 / (m - i)])
                ctxt_a = heaan.Ciphertext()
                ctxt_b = heaan.Ciphertext()
                self.encryptor.encrypt(a, self.enc_key, ctxt_a)
                self.encryptor.encrypt(b, self.enc_key, ctxt_b)
                # gini_left = 1.0 - sum(
                #     (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                # )
                # gini_right = 1.0 - sum(
                #     (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                # )
                gini_left = self.Mysub(ctxt_flag, self.sub_gini(num_left, ctxt_a))
                left = d(gini_left, self.decryptor, self.secret_key)
                gini_right = self.Mysub(ctxt_flag, self.sub_gini(num_right, ctxt_b))
                right = d(gini_right, self.decryptor, self.secret_key)
                # print("gini_left: ", d(gini_left, self.decryptor, self.secret_key))
                # print("gini_right: ", d(gini_right, self.decryptor, self.secret_key))
                
                # gini = (i * gini_left + (m - i) * gini_right) / m
                gini = self.cal_gini(i, m, m_1, gini_left, gini_right)
                g = d(gini, self.decryptor, self.secret_key)
                # print(d(gini, self.decryptor, self.secret_key))
                
                # (both have to end up on the same side of a split).
                if self.equal(thresholds[i], thresholds[i - 1]):
                    continue
                # print(self.compare(gini, best_gini))
                if self.compare(gini, best_gini):
                    best_gini = gini
                    best_idx = idx
                    best_thr = self.midpoint(thresholds[i], thresholds[i - 1])

        return best_idx, best_thr

    def compare_thr(self, X, thr, idx):
        result = []
        for i in X:
            # result.append(i[idx])
            result.append(np.array(d(i[idx], self.decryptor, self.secret_key)) <
                          d(thr, self.decryptor, self.secret_key))

        return result

    def n_class(self, enc_label):
        fl = dec_arr(enc_label, self.decryptor, self.secret_key)
        a = []
        for i in fl:
            a.append(int(np.array(i).real))

        return set(a)

    def divide(self, indices_left, X, y):
        X_left = []
        y_left = []
        X_right = []
        y_right = []
        for i in range(len(X)):
            if indices_left[i]:
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                X_right.append(X[i])
                y_right.append(y[i])

        return X_left, y_left, X_right, y_right

    def fit(self, X, y):
        self.n_classes_ = len(self.n_class(y))  # classes are assumed to go from 0 to n-1
        self.n_features_ = len(X[0])
        # self._best_split(X, y)
        self.tree_ = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = self.enc_y(y)
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(
            gini=y,
            num_samples=len(y),
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )

        # Split recursively until maximum depth is reached.
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            # print(d(thr, self.decryptor, self.secret_key))
            if idx is not None:
                indices_left = self.compare_thr(X, thr, idx)
                # indices_left = X[:, idx] < thr
                X_left, y_left, X_right, y_right = self.divide(indices_left, X, y)
                # X_left, y_left = X[indices_left], y[indices_left]
                # X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)

        return node

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _predict(self, inputs):
        """Predict class for a single sample."""
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < np.array(d(node.threshold, self.decryptor, self.secret_key)):
                node = node.left
            else:
                node = node.right
        return node.predicted_class

## Read and split data

In [ ]:
def data_pre():
    data = pd.read_csv('/Users/xialetian/Downloads/Iriss.csv')
    columns = data.columns.values
    le_species = LabelEncoder()
    data['label'] = le_species.fit_transform(data[columns[5]])
    data = data.drop(columns[5], axis='columns')
    data = data.drop(columns[0], axis='columns')
    y = data['label'].to_numpy()
    X = data.loc[:, data.columns != 'label'].to_numpy()
    train_data, test_data, train_label, test_label = train_test_split(X,y)

    return train_data, test_data, train_label, test_label

## Preparation before encryption

- context
- secret_key
- encryptor
- evaluator
- keypack/public_key/enc_key
- decryptor

In [ ]:
params = heaan.Parameters()
log_slots = 16
context = heaan.Context(params)
context.make_bootstrappable(log_slots)
key_dir_path = "venv/keys"
secret_key = heaan.SecretKey(context)
keypack = heaan.PublicKeyPack(context, secret_key, key_dir_path)
eval = heaan.HomEvaluator(context)
encryptor = heaan.Encryptor(context)
enc_key = keypack.get_enc_key()
decryptor = heaan.Decryptor(context)

# train_data, test_data, train_label, test_label = data_pre()
# X = train_data
# y = train_label
# enc_data, enc_label = data_enc(X, y, enc_key, encryptor, context)

# df = pd.read_csv("/Users/xialetian/Downloads/wifi_localization.txt", delimiter="\t")
# Fit data.
# clf = DecisionTreeClassifier(encryptor, enc_key, eval, keypack, decryptor, secret_key, context, max_depth=5)
# clf.fit(enc_data, enc_label)
# result = clf.predict(test_data)

## Encrypted data & label

In [ ]:
# train_data, test_data, train_label, test_label = data_pre()
X = train_data
y = train_label
enc_data, enc_label = data_enc(X, y, enc_key, encryptor, context)
enc_data

## Train& Test the encryption model

In [ ]:
clf = DecisionTreeClassifier(encryptor, enc_key, eval, keypack, decryptor, secret_key, context, max_depth=5)
clf.fit(enc_data, enc_label)
result = clf.predict(test_data)

## Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
c_matrix = confusion_matrix(test_label, result)
c_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
# fig = plt.figure()
plt.figure(dpi=100)
# ax = fig.add_subplot(111)
conf_mat = c_matrix
sns.heatmap(conf_mat, annot=True, cmap="BuPu")
# ax.set_title('confusion matrix') 
# ax.set_xlabel('predict') 
# ax.set_ylabel('true') 
plt.show()

In [ ]:
print(classification_report(test_label, result))

- **Precision**: Precision (also called positive predictive value) is the fraction of relevant instances among the retrieved instances
- **Recall**: recall (also known as sensitivity) is the fraction of relevant instances that were retrieved
- **f1-score**: The F-score, also called the F1-score, is a measure of a model's accuracy on a dataset

In [ ]:
print(context)

## Sklearn  plaintext model

In [ ]:
from sklearn import tree

In [ ]:
dt = tree.DecisionTreeClassifier()
dt.fit(train_data, train_label)

In [ ]:
result_ = dt.predict(test_data)

In [ ]:
d_matrix = confusion_matrix(test_label, result_)
d_matrix

In [ ]:
sns.set()
# fig = plt.figure()
plt.figure(dpi=100)
# ax = fig.add_subplot(111)
conf_mat = d_matrix
sns.heatmap(conf_mat, annot=True, cmap="Blues")
# ax.set_title('confusion matrix') 
# ax.set_xlabel('predict') 
# ax.set_ylabel('true') 
plt.show()

In [ ]:
print(classification_report(test_label, result_))